In [2]:
import cv2 as cv
import numpy as np

def fftshift(img):
    h, w = img.shape
    cx, cy = w // 2, h // 2
    q0 = img[0:cy, 0:cx]
    q1 = img[0:cy, cx:w]
    q2 = img[cy:h, 0:cx]
    q3 = img[cy:h, cx:w]

    tmp = np.copy(q0)
    img[0:cy, 0:cx] = q3
    img[cy:h, cx:w] = tmp

    tmp = np.copy(q1)
    img[0:cy, cx:w] = q2
    img[cy:h, 0:cx] = tmp
    return img

def filter2DFreq(input_img, H):
    planes = [np.float32(input_img), np.zeros_like(input_img, dtype=np.float32)]
    complexI = cv.merge(planes)
    complexI = cv.dft(complexI, flags=cv.DFT_SCALE)

    planesH = [np.float32(H), np.zeros_like(H, dtype=np.float32)]
    complexH = cv.merge(planesH)
    complexIH = cv.mulSpectrums(complexI, complexH, 0)

    complexIH = cv.idft(complexIH)
    output_img = cv.split(complexIH)[0]
    return output_img

def synthesizeFilterH(H, center, radius):
    h, w = H.shape
    c2 = (center[0], h - center[1])
    c3 = (w - center[0], center[1])
    c4 = (w - center[0], h - center[1])
    for c in [center, c2, c3, c4]:
        cv.circle(H, c, radius, 0, -1)

def calcPSD(input_img, log=False):
    planes = [np.float32(input_img), np.zeros_like(input_img)]
    complexI = cv.merge(planes)
    complexI = cv.dft(complexI)
    planes = cv.split(complexI)

    planes[0][0, 0] = 0
    planes[1][0, 0] = 0

    imgPSD = cv.magnitude(planes[0], planes[1])
    imgPSD = imgPSD ** 2

    if log:
        imgPSD = np.log(imgPSD + 1)

    return imgPSD

def main():
    filename = 'noise.jpg'
    imgIn = cv.imread(filename, cv.IMREAD_GRAYSCALE)
    if imgIn is None:
        print("ERROR: Image cannot be loaded.")
        return

    cv.imshow("Image corrupted", imgIn)

    imgIn = np.float32(imgIn)
    h, w = imgIn.shape
    imgIn = imgIn[0:h & -2, 0:w & -2]

    imgPSD = calcPSD(imgIn)
    imgPSD = fftshift(imgPSD)
    imgPSD = cv.normalize(imgPSD, None, 0, 255, cv.NORM_MINMAX)

    H = np.ones_like(imgIn, dtype=np.float32)
    r = 21
    synthesizeFilterH(H, (705, 458), r)
    synthesizeFilterH(H, (850, 391), r)
    synthesizeFilterH(H, (993, 325), r)

    H = fftshift(H)
    imgOut = filter2DFreq(imgIn, H)
    imgOut = np.uint8(cv.normalize(imgOut, None, 0, 255, cv.NORM_MINMAX))

    cv.imwrite("result.jpg", imgOut)
    cv.imwrite("PSD.jpg", np.uint8(imgPSD))
    H = fftshift(H)
    H = cv.normalize(H, None, 0, 255, cv.NORM_MINMAX)
    cv.imwrite("filter.jpg", H)

    cv.imshow("Debluring", imgOut)
    cv.waitKey(0)
    cv.destroyAllWindows()

if __name__ == "__main__":
    main()
